# American Community Survey

This example uses the Public Use Microdata Sample provided by the Census Bureau. You can read about the [data documentation](https://www.census.gov/programs-surveys/acs/microdata/documentation.html) or view the [data transformation](https://github.com/jaanli/exploring_american_community_survey_data/blob/main/american_community_survey/models/public_use_microdata_sample/generated/enum_types_mapped_renamed/housing_units_united_states_first_tranche_enum_mapped_renamed.sql).

![](https://s13.gifyu.com/images/SCGH2.gif)

In [1]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
from ipywidgets import FloatRangeSlider, jsdlink
from palettable.colorbrewer.diverging import BrBG_10

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
from lonboard.controls import MultiRangeSlider
from lonboard.layer_extension import DataFilterExtension

In [3]:
!wget -O 2020_census_microdata_tiger_shapefile.zip https://data.payless.health/census.gov%2Famerican_community_survey%2F2020_census_microdata_tiger_shapefile.zip
!unzip 2020_census_microdata_tiger_shapefile.zip

--2024-02-14 07:52:51--  https://data.payless.health/census.gov%2Famerican_community_survey%2F2020_census_microdata_tiger_shapefile.zip
Resolving data.payless.health (data.payless.health)... 18.164.116.95, 18.164.116.105, 18.164.116.102, ...
Connecting to data.payless.health (data.payless.health)|18.164.116.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68488320 (65M) [application/zip]
Saving to: ‘2020_census_microdata_tiger_shapefile.zip’

2020_census_microda 100%[===================>]  65.32M  48.8MB/s    in 1.3s    

2024-02-14 07:52:52 (48.8 MB/s) - ‘2020_census_microdata_tiger_shapefile.zip’ saved [68488320/68488320]

Archive:  2020_census_microdata_tiger_shapefile.zip
 extracting: 2020_census_microdata_tiger_shapefile.cpg  
  inflating: 2020_census_microdata_tiger_shapefile.dbf  
  inflating: 2020_census_microdata_tiger_shapefile.shp  
  inflating: 2020_census_microdata_tiger_shapefile.shx  


In [2]:
from pathlib import Path
import pandas as pd

url_first = "https://data.payless.health/census.gov%2Famerican_community_survey%2Fpublic_use_microdata%2F2022_acs_pums_individual_people_united_states_first_tranche.parquet"
url_second = "https://data.payless.health/census.gov%2Famerican_community_survey%2Fpublic_use_microdata%2F2022_acs_pums_individual_people_united_states_second_tranche.parquet"
local_path = Path("2022_acs_pums_individual_people_united_states.parquet")

# Check if the local file exists to avoid re-downloading
columns_of_interest = ["Public use microdata area code (PUMA) based on 2020 Census definition (areas with population of 100,000 or more, use with ST for unique code)", 
                       "Total person's income (use ADJINC to adjust to constant dollars)"]
if not local_path.exists():
    df_first = pd.read_parquet(url_first, columns=columns_of_interest)
    df_second = pd.read_parquet(url_second, columns=columns_of_interest)
    df = pd.concat([df_first, df_second])

    df.rename(columns={"Public use microdata area code (PUMA) based on 2020 Census definition (areas with population of 100,000 or more, use with ST for unique code)": "puma",
                        "Total person's income (use ADJINC to adjust to constant dollars)": "income"}, inplace=True)
    df.to_parquet(local_path)
else:
    df = pd.read_parquet(local_path)


shapefile_path = "2020_census_microdata_tiger_shapefile.shp"
puma_shapes = gpd.read_file(shapefile_path)
puma_shapes = puma_shapes.rename(columns={'PUMACE10': "puma"})
puma_shapes['centroid'] = shapely.centroid(puma_shapes['geometry'])

len(puma_shapes['puma'].unique()), len(df['puma'].unique())
missing_pumas = set(df['puma'].unique()) - set(puma_shapes['puma'].unique())
print('missing pumas', len(missing_pumas))
print('missing pumas', missing_pumas)

# drop missing pumas
df = df[~df['puma'].isin(missing_pumas)]
merged_df = df.merge(puma_shapes[["puma", "centroid"]], on="puma",
                      how='left')
# filter for nan values
idx = ~merged_df['centroid'].isna() & ~merged_df['income'].isna()
df['income'] = df["income"].astype(float)
gdf = gpd.GeoDataFrame(df["income"][idx], geometry=merged_df["centroid"][idx])

print('length', len(gdf))
gdf.head()

missing pumas 440
missing pumas {'01398', '02407', '20604', '04209', '03216', '81003', '00706', '01609', '19701', '08522', '08520', '16700', '07709', '25305', '00606', '35013', '17900', '04411', '02011', '07119', '03227', '26701', '06516', '54001', '07500', '03780', '23315', '07705', '00607', '03156', '11302', '04311', '03771', '04207', '20902', '01311', '25302', '02804', '03770', '16500', '03781', '06520', '06517', '01909', '08516', '03231', '06524', '04705', '04263', '15303', '03110', '20703', '21100', '04402', '10903', '04408', '03406', '27702', '02406', '07513', '02012', '15302', '06720', '07509', '04318', '21104', '02806', '09900', '20502', '68701', '22000', '03223', '04205', '26500', '01611', '07707', '23318', '05105', '02405', '26702', '04210', '35017', '07100', '61001', '23314', '03154', '27701', '00609', '23311', '09200', '07508', '07328', '05926', '21000', '01605', '02105', '25301', '01512', '05925', '81001', '06521', '08521', '60001', '26302', '04413', '21404', '07200', '202

,income,geometry
0,18800.0,POINT (-75.47555 40.59609)
1,12500.0,POINT (-86.22214 32.34156)
2,16400.0,POINT (-93.72120 33.64495)
3,8600.0,POINT (-96.12896 42.56285)
4,5000.0,POINT (-85.39690 40.22753)


In [4]:
import geopandas as gpd
import pandas as pd

# Assuming 'puma_shapes' is your GeoDataFrame with PUMA geometries
# and 'df' is your DataFrame with 'puma' codes and corresponding data

# Calculate the count of records for each PUMA code in 'df'
puma_counts = df['puma'].value_counts().rename_axis('puma').reset_index(name='count')

# Merge the counts with the PUMA shapes
num_people_in_area = puma_shapes.merge(puma_counts, on='puma', how='left')

# Ensure the 'count' column is filled with zeros where there are NaNs to avoid errors
num_people_in_area['count'] = num_people_in_area['count'].fillna(0).astype(int)

# Now, use the sample_points method on the whole GeoSeries, passing the 'count' column as sizes
# Note: Ensure all geometries are valid and count > 0 to sample, otherwise handle accordingly
sampled_points = num_people_in_area['geometry'].sample_points(size=num_people_in_area['count'], seed=42)


/var/folders/j6/7vrpplt53hs3z_nyd8j0zqsh0000gn/T/ipykernel_24676/1514105908.py:18: FutureWarning: The 'seed' keyword is deprecated. Use 'rng' instead.
  sampled_points = num_people_in_area['geometry'].sample_points(size=num_people_in_area['count'], seed=42)
/Users/me/jaanli/lonboard/.venv/lib/python3.11/site-packages/geopandas/tools/_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
/Users/me/jaanli/lonboard/.venv/lib/python3.11/site-packages/geopandas/tools/_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
/Users/me/jaanli/lonboard/.venv/lib/python3.11/site-packages/geopandas/tools/_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future,

In [8]:
import numpy as np
import pandas as pd
import geopandas as gpd

# Exploded sampled points as individual geometries
points_for_people = sampled_points.explode().reset_index(drop=True)

# Assuming 'df' has a 'puma' column with PUMA codes as strings
# and you've already prepared points_for_people as a GeoSeries of points from the exploded sampled_points

# Group the exploded points by PUMA and accumulate indices
# First, ensure there's a way to map each point to its PUMA. This might require an additional step
# if points_for_people doesn't inherently know its PUMA. Assuming we have a mapping:
puma_to_point_index = {puma: [] for puma in df['puma'].unique()}

# If you had a direct way to map each exploded point to its PUMA, you'd fill puma_to_point_index here
# For demonstration, let's assume each point in points_for_people is already associated with a PUMA:
for i, puma in enumerate(puma_indices):  # This assumes puma_indices is aligned with points_for_people
    puma_to_point_index[puma].append(i)

# Now you have a mapping of PUMA to list of indices of points
# Shuffle the indices for each PUMA to ensure random assignment
for puma in puma_to_point_index:
    np.random.shuffle(puma_to_point_index[puma])

# Assign points to individuals in 'df' based on their PUMA
def assign_point(puma):
    if puma_to_point_index[puma]:
        return puma_to_point_index[puma].pop()  # This pops an index, ensuring it's used only once
    else:
        return None  # Or handle the case where no points are left for the PUMA

# Use apply to assign a point index to each row in df based on its PUMA
df['point_index'] = df['puma'].apply(assign_point)

# Map the point geometries to df using the assigned indices
df['geometry'] = df['point_index'].apply(lambda x: points_for_people.iloc[x] if pd.notnull(x) else None)

# Convert df to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.crs = sampled_points.crs  # Assuming CRS needs setting


/var/folders/j6/7vrpplt53hs3z_nyd8j0zqsh0000gn/T/ipykernel_24676/1410429490.py:6: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  points_for_people = sampled_points.explode().reset_index(drop=True)


In [16]:
gdf.head()

,puma,income,point_index,geometry
0,02803,18800.0,22342836,POINT (-75.50154 40.55573)
1,02000,12500.0,21400851,POINT (-81.11718 35.93057)
2,02803,16400.0,22344187,POINT (-75.47385 40.60554)
3,01100,8600.0,1311981,POINT (-85.63642 33.88719)
4,01502,5000.0,25778226,POINT (-95.30144 32.37114)


In [10]:
filter_extension = DataFilterExtension(filter_size=1)

In [11]:
# If you want to define specific bounds for normalization:
min_bound = 5000  # Example minimum income
max_bound = 100000  # Example maximum income

normalized_income = (gdf["income"].values.astype(float) - min_bound) / (max_bound - min_bound)

fill_color = apply_continuous_cmap(normalized_income, BrBG_10)

radius = normalized_income * 200  # Adjust 200 as per your visualization needs

In [12]:
filter_values = gdf["income"]

initial_filter_range = [5_000, 100_000]


In [13]:
layer = ScatterplotLayer.from_geopandas(
    gdf,
    extensions=[filter_extension],
    get_fill_color=fill_color,
    get_radius=radius,
    get_filter_value=filter_values,
    filter_range=initial_filter_range,
    radius_units="meters",
    radius_min_pixels=1,
)
m = Map(layer)
m

/Users/me/jaanli/lonboard/.venv/lib/python3.11/site-packages/lonboard/_utils.py:57: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col_name] = pd.to_numeric(
/Users/me/jaanli/lonboard/.venv/lib/python3.11/site-packages/lonboard/_utils.py:64: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col_name] = pd.to_numeric(


Map(layers=[ScatterplotLayer(extensions=[DataFilterExtension()], filter_range=[5000.0, 100000.0], get_fill_col…

In [14]:
income_slider = FloatRangeSlider(
    value=initial_filter_range,
    min=0,
    max=100_000,
    step=1,
    description="Income: ",
)


multi_slider = MultiRangeSlider([income_slider])
multi_slider

MultiRangeSlider(children=(FloatRangeSlider(value=(5000.0, 100000.0), description='Income: ', max=100000.0, st…

In [15]:
_ = jsdlink((income_slider, "value"), (layer, "filter_range"))